In [1]:
from rdflib import Graph, Literal, RDF, URIRef
# rdflib knows about quite a few popular namespaces, like W3C ontologies, schema.org etc.
import rdflib
import pandas as pd
from rdflib import Namespace

In [2]:
# ouvrir l'ontologie en tant que graphe avec rdflib

g = Graph() # création d'un graphe vide / remise à zéro
# On ouvre/parse le fichier de rdf avec toutes les classes de notre master ontologie 
# Le parse comme le html ta mère
g.parse("../Ontologie_MasterTAL.rdf", format='xml')

# print(g.serialize(format='turtle')) # print en format turtle


# On crée un namespace (ex: foaf, rdf, rdfs, etc) pour notre ontologie
# Son URI est l'URI de notre ontologie
MT = Namespace("http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#")
# On bind notre namespace à notre graphe pour dire il y a des trucs qu'on va mettre dedans
g.bind("mt", MT)      # bind a user-declared namespace to a prefix


print("--- printing mboxes ---")
print(g.serialize(format='n3'))
print("--- printing xml format ---")
print(g.serialize(format='xml'))

--- printing mboxes ---
@prefix mt: <http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL> a owl:Ontology .

mt:Alternance a owl:Class ;
    rdfs:subClassOf mt:M2 .

mt:CDD a owl:Class ;
    rdfs:subClassOf mt:Emploi .

mt:CDI a owl:Class ;
    rdfs:subClassOf mt:Emploi .

mt:ContratAlternance a owl:Class ;
    rdfs:subClassOf mt:ExperienceTAL .

mt:DevoirMaisonGroupe a owl:Class ;
    rdfs:subClassOf mt:Evaluation .

mt:DevoirMaisonSolo a owl:Class ;
    rdfs:subClassOf mt:Evaluation .

mt:DevoirTable a owl:Class ;
    rdfs:subClassOf mt:Evaluation .

mt:IM a owl:Class ;
    rdfs:subClassOf mt:M2 .

mt:M1 a owl:Class ;
    rdfs:subClassOf mt:Parcours .

mt:OralGroupe a owl:Class ;
    rdfs:subClassOf mt:Evaluation .

mt:OralSolo a owl:Class ;
    rdfs

# Ouvrir les dataframes 

In [3]:
formulaire = pd.read_csv("../data/formulaire_cleaned.csv")
df_etudiant = pd.read_csv("../data/eleves_avec_cours.csv")
df_cours = pd.read_csv("../data/df_cours.csv")
df_prof = pd.read_csv("../data/df_prof.csv")
df_stage= pd.read_csv("../data/df_stages.csv")


In [4]:
dico_evaluation = {'DevoirMaisonGroupe': "Devoir maison (en groupe)", 
                   'DevoirTable': "Devoir sur table", 
                   'DevoirMaisonSolo': "Devoir maison (en solo)", 
                   'ProjetGroupe': 'Projet (en groupe)',
                   'ProjetSolo':'Projet (solo)',
                   'OralGroupe': 'Présentation orale (en groupe)',
                   'OralSolo': 'Présentation orale (solo)'
                   }
dico_reverse_eval = {v: k for k, v in dico_evaluation.items()}
print(dico_reverse_eval)


{'Devoir maison (en groupe)': 'DevoirMaisonGroupe', 'Devoir sur table': 'DevoirTable', 'Devoir maison (en solo)': 'DevoirMaisonSolo', 'Projet (en groupe)': 'ProjetGroupe', 'Projet (solo)': 'ProjetSolo', 'Présentation orale (en groupe)': 'OralGroupe', 'Présentation orale (solo)': 'OralSolo'}


In [5]:
dico_parcours = {"nom_parcours": ["M1_Nanterre_23-24", "M1_Inalco_23-24", "M1_Sorbonne_23-24", "M2_RD_Nanterre_24-25", "M2_RD_Inalco_24-25", "M2_RD_Sorbonne_24-25", "M2_Alterance_24-25", "M2_IM_24-25", "M2_TetraDom_24-25"],
                 "etablissement": ["Nanterre", "Inalco", "SorbonneNouvelle", "Nanterre", "Inalco", "SorbonneNouvelle", "Nanterre", "Inalco", "Inalco"],
                 "liste_eleves" : [[], [], [], [], [], [], [], [], []],
                 "annee" : ["2023-2024", "2023-2024", "2023-2024", "2024-2025", "2024-2025", "2024-2025", "2024-2025", "2024-2025", "2024-2025"],}

# liste eleves la relation sera créée dans l'ontologie automatiquement grace à la relation inverse (éléves -> parcours) déjà dans la df élèves
df_parcours = pd.DataFrame(dico_parcours)
df_parcours.to_csv("../data/dataframe_parcours.csv", index=False)


# Ajout des triplets 

In [6]:
def ajout_etudiant(df_etudiant, g, dico_reverse_eval, df_cours):
	'''On ajoute les etudiants à notre ontologie'''
	for row_eleve in df_etudiant.itertuples(index=False, name = 'Eleve'):
		# On crée un URI pour chaque étudiant
		etudiant = URIRef(MT + (row_eleve.id).replace(" ", ""))
		# On ajoute l'étudiant à notre ontologie
		g.add((etudiant, RDF.type, MT.Eleve))
		g.add((etudiant, MT.nom, Literal(row_eleve.nom)))
		g.add((etudiant, MT.prenom, Literal(row_eleve.prenom)))
		g.add((etudiant, MT.genre, Literal(row_eleve.genre)))
		g.add((etudiant, MT.dateNaissance, Literal(row_eleve.date_naissance)))
		g.add((etudiant, MT.nationalite, Literal(row_eleve.nationalite)))
		
		if row_eleve.redoublement == "non":
			g.add((etudiant, MT.redoublement, Literal(False)))
		else:
			g.add((etudiant, MT.redoublement, Literal(True)))
		
		if row_eleve.tiers_temps == "non":
			g.add((etudiant, MT.tiersTemps, Literal(False)))
		else: 
			g.add((etudiant, MT.tiersTemps, Literal(True)))

		# Parcours M1 
		if "Nanterre" in row_eleve.inscription_M1:
			g.add((etudiant, MT.inscritA, URIRef(MT +"M1_Nanterre_23-24")))
		elif "INALCO" in row_eleve.inscription_M1:
			g.add((etudiant, MT.inscritA, URIRef(MT+"M1_Inalco_23-24")))
		elif "Sorbonne" in row_eleve.inscription_M1:
			g.add((etudiant, MT.inscritA, URIRef(MT+"M1_Sorbonne_23-24")))
		# Parcours M2
		if "Alternance" in row_eleve.parcours_M2: 
			g.add((etudiant, MT.inscritA, URIRef(MT+"M2_Alterance_24-25")))
		elif "Ingénierie Multilingue" in row_eleve.parcours_M2:
			g.add((etudiant, MT.inscritA, URIRef(MT+"M2_IM_24-25")))
		elif "TetraDom" in row_eleve.parcours_M2:
			g.add((etudiant, MT.inscritA, URIRef(MT+"M2_TetraDom_24-25")))
		elif "Recherche et Développement (Nanterre)" in row_eleve.parcours_M2:
			g.add((etudiant, MT.inscritA, URIRef(MT+"M2_RD_Nanterre_24-25")))
		elif "Recherche et Développement (INALCO)" in row_eleve.parcours_M2:
			g.add((etudiant, MT.inscritA, URIRef(MT+"M2_RD_Inalco_24-25")))
		elif "Recherche et Développement (Sorbonne Nouvelle)" in row_eleve.parcours_M2:
			g.add((etudiant, MT.inscritA, URIRef(MT+"M2_RD_Sorbonne_24-25")))

		if pd.notna(row_eleve.inscription_simultanee):
			g.add((etudiant, MT.inscritAussiA, Literal(row_eleve.inscription_simultanee)))
		if pd.notna(row_eleve.BAC):
			g.add((etudiant, MT.bac, Literal(row_eleve.BAC)))
		if pd.notna(row_eleve.licence):
			g.add((etudiant, MT.licence, Literal(row_eleve.licence)))
		if pd.notna(row_eleve.mention_licence):
			g.add((etudiant, MT.mention_licence, Literal(row_eleve.mention_licence)))
		if pd.notna(row_eleve.autre_master):
			g.add((etudiant, MT.autre_master, Literal(row_eleve.autre_master)))
		if pd.notna(row_eleve.mention_master_precedent):
			g.add((etudiant, MT.mention_master, Literal(row_eleve.mention_master_precedent)))
		if pd.notna(row_eleve.autres_diplomes):
			g.add((etudiant, MT.autre_diplome, Literal(row_eleve.autres_diplomes)))
		if pd.notna(row_eleve.job_etudiant):
			g.add((etudiant, MT.jobEtudiant, Literal(row_eleve.job_etudiant)))
		if pd.notna(row_eleve.profession_avant):
			g.add((etudiant, MT.profession_avant, Literal(row_eleve.profession_avant)))

		if pd.notna(row_eleve.prefere_sujet):
			g.add((etudiant, MT.sujet_pref, Literal(row_eleve.prefere_sujet)))
		
		# Boucle avec les types d'évaluations
		if pd.notna(row_eleve.prefere_evaluation):
			evals = (row_eleve.prefere_evaluation).split(', ')
			for eval in evals: 
				if eval in dico_reverse_eval.keys():
					g.add((etudiant, MT.prefereCommeEvaluation, URIRef(MT + dico_reverse_eval[eval])))
				else: 
					g.add((etudiant, MT.prefereCommeEvaluation, URIRef(MT + (eval).replace(" ", "_"))))
				   
		# Boucle avec les noms des cours pref + recup de l'id de k dans df_cours
		if pd.notna(row_eleve.prefere_cours):
			cours = (row_eleve.prefere_cours).replace('[', '').replace(']','').split(', ')
			for c in cours:
				if c.strip(" ") in df_cours['nom_cours'].values: 
					# print(df_cours.loc[df_cours['nom_cours'] == c.strip(" "), 'id_cours'].iloc[0])
					id_cours = df_cours.loc[df_cours['nom_cours'] == c.strip(" "), 'id_cours'].iloc[0]
					g.add((etudiant, MT.prefereLeCoursDe, URIRef(MT + id_cours)))
				else:
					print(c) # g.add((etudiant, MT.prefereLeCoursDe, Literal(c)))
		
		if pd.notna(row_eleve.suggestion_maquette):
			g.add((etudiant, MT.suggestion_maquette, Literal(row_eleve.suggestion_maquette)))
		
		if pd.notna(row_eleve.ennemi_jure):
			for copain in row_eleve.ennemi_jure.replace("[", '').replace("]", '').split(', '):
				g.add((etudiant, MT.aCommeEnnemi, URIRef(MT + copain)))

		if pd.notna(row_eleve.sante_avant):
			g.add((etudiant, MT.sante_avant, Literal(int(row_eleve.sante_avant))))
		if pd.notna(row_eleve.sante_apres):
			g.add((etudiant, MT.sante_apres, Literal(int(row_eleve.sante_apres))))
		if pd.notna(row_eleve.quota_pleure):
			g.add((etudiant, MT.quota_pleure, Literal(row_eleve.quota_pleure)))
		if pd.notna(row_eleve.heures_sommeil):
			g.add((etudiant, MT.temps_sommeil, Literal(int(row_eleve.heures_sommeil))))
		
		if pd.notna(row_eleve.difficulte_M1):
			g.add((etudiant, MT.difficulte_M1, Literal(int(row_eleve.difficulte_M1))))
		
		if pd.notna(row_eleve.difficulte_alternance):
			g.add((etudiant, MT.difficulte_M2, Literal(int(row_eleve.difficulte_alternance))))
		if pd.notna(row_eleve.difficulte_IM):
			g.add((etudiant, MT.difficulte_M2, Literal(int(row_eleve.difficulte_IM))))
		if pd.notna(row_eleve.difficulte_RD):
			g.add((etudiant, MT.difficulte_M2, Literal(int(row_eleve.difficulte_RD))))
		if pd.notna(row_eleve.difficulte_tetradom):
			g.add((etudiant, MT.difficulte_M2, Literal(int(row_eleve.difficulte_tetradom))))

		if pd.notna(row_eleve.jeter_linux):
			g.add((etudiant, MT.relation_linux, Literal(row_eleve.jeter_linux))) 
		if pd.notna(row_eleve.jeter_mac):
			g.add((etudiant, MT.relation_mac, Literal(row_eleve.jeter_mac))) 

		if pd.notna(row_eleve.conseil_eleve):
			g.add((etudiant, MT.conseil_eleve, Literal(row_eleve.conseil_eleve)))

		if pd.notna(row_eleve.conseil_prof):
			g.add((etudiant, MT.conseil_prof, Literal(row_eleve.conseil_prof)))

		if pd.notna(row_eleve.recommandation):
			g.add((etudiant, MT.recommendation, Literal(row_eleve.recommandation)))

		liste_semestre = [row_eleve.normalised_m1_s1_courses, row_eleve.normalised_m1_s1_optionnal,row_eleve.normalised_m1_s1_extra,row_eleve.normalised_m1_s2_courses,row_eleve.normalised_m1_s2_extra,row_eleve.normalised_m2_s1_courses,row_eleve.normalised_m2_s1_extra]
		
		# Boucle pour ajouter les élèves aux cours 
		for semestre in liste_semestre:
			if pd.notna(semestre): 
				cours = (semestre).replace('[', '').replace(']','').split(', ')
				# print(cours)
				for c in cours:
					if c != '':
						# print(c)
						if c.strip("'").strip('"') in df_cours['nom_cours'].values: 
							# print(df_cours.loc[df_cours['nom_cours'] == c.strip("'").strip('"'), 'id_cours'].iloc[0])
							id_cours = df_cours.loc[df_cours['nom_cours'] == c.strip("'").strip('"'), 'id_cours'].iloc[0]
							g.add((etudiant, MT.suitLeCoursDe, URIRef(MT + id_cours)))
						else:
							print(f"ET MERDE ON A OUBLIE : {c}") # g.add((etudiant, MT.prefereLeCoursDe, Literal(c)))
			

	# print("--- printing mboxes ---")
	# print(g.serialize(format='n3'))
	# print("--- printing xml format ---")
	# print(g.serialize(format='xml'))

	return g

g = ajout_etudiant(df_etudiant, g, dico_reverse_eval, df_cours)

In [7]:
def ajout_stage (df_stage, g):
	'''On ajoute les stages à notre ontologie'''
	for row_stage in df_stage.itertuples(index=False, name = 'Stage'):
		etudiant = URIRef(MT + (row_stage.id_eleves).replace(" ", ""))
		if pd.notna(row_stage.id_m_1):
			stage1 = URIRef(MT + (row_stage.id_m_1).replace(" ", ""))
			g.add((etudiant, MT.aCommeExperience, stage1))
			g.add((stage1, RDF.type, MT.Stage))
			if pd.notna(row_stage.stages_m1_duree):
				g.add((stage1, MT.duree, Literal(int(row_stage.stages_m1_duree.strip(" mois")))))
			if pd.notna(row_stage.stages_m1_nom_orga):
				g.add((stage1, MT.nomOrganismeAcceuil, Literal(row_stage.stages_m1_nom_orga)))
			if pd.notna(row_stage.stages_m1_type_orga):
				g.add((stage1, MT.typeOrganisme, Literal(row_stage.stages_m1_type_orga)))
			if pd.notna((row_stage.stages_m1_note))or row_stage.stages_m1_note != None:
				g.add((stage1, MT.note_experience, Literal(int(row_stage.stages_m1_note))))
			if pd.notna(row_stage.stages_m1_sujet):
				g.add((stage1, MT.sujetMissions, Literal(row_stage.stages_m1_sujet)))
			if pd.notna(row_stage.stages_m1_tuteur):
				nom_prenom = row_stage.stages_m1_tuteur.strip(" ").split(" ") 
				tuteur = nom_prenom[0] + "_" + "_".join([nom.upper() for nom in nom_prenom[1:]])
				print(tuteur)
				g.add((etudiant, MT.aPourTuteurDeStage, URIRef(MT + tuteur)))


		if pd.notna(row_stage.id_m_2):
			stage2 = URIRef(MT + (row_stage.id_m_2).replace(" ", ""))
			g.add((etudiant, MT.aCommeExperience, stage2))
			g.add((stage2, RDF.type, MT.Stage))
			if pd.notna(row_stage.stages_m2_duree):
				g.add((stage2, MT.duree, Literal(int(row_stage.stages_m2_duree.strip(" mois")))))
			if pd.notna(row_stage.stages_m2_nom_orga):
				g.add((stage2, MT.nomOrganismeAcceuil, Literal(row_stage.stages_m2_nom_orga)))
			if pd.notna(row_stage.stages_m2_type_orga):
				g.add((stage2, MT.typeOrganisme, Literal(row_stage.stages_m2_type_orga)))
			if pd.notna((row_stage.stages_m2_note)):
				g.add((stage2, MT.note_experience, Literal(int(row_stage.stages_m2_note))))
			if pd.notna(row_stage.stages_m2_sujet):
				g.add((stage2, MT.sujetMissions, Literal(row_stage.stages_m2_sujet)))
			if pd.notna(row_stage.stages_m2_tuteur):
				nom_prenom = row_stage.stages_m2_tuteur.strip(" ").split(" ") 
				tuteur = nom_prenom[0] + "_" + "_".join([nom.upper() for nom in nom_prenom[1:]])
				print(tuteur)
				g.add((etudiant, MT.aPourTuteurDeStage, URIRef(MT + tuteur)))
		
		if pd.notna(row_stage.id_m_2_alt_alt):
			stage3 = URIRef(MT + (row_stage.id_m_2_alt_alt).replace(" ", ""))
			g.add((etudiant, MT.aCommeExperience, stage3))
			g.add((stage3, RDF.type, MT.ContratAlternance))

			if pd.notna(row_stage.stages_m2_alternance_type_org_alt):
				g.add((stage3, MT.nomOrganismeAcceuil, Literal(row_stage.stages_m2_alternance_type_org_alt)))
			
			if pd.notna(row_stage.stages_m2_alternance_missions_alt):
				g.add((stage3, MT.sujetMissions, Literal(row_stage.stages_m2_alternance_missions_alt)))
			

		if pd.notna(row_stage.id_m_2_alt_stage):
			stage3 = URIRef(MT + (row_stage.id_m_2_alt_stage).replace(" ", ""))
			g.add((etudiant, MT.aCommeExperience, stage3))
			g.add((stage3, RDF.type, MT.Stage))

			if pd.notna(row_stage.stages_m2_alternance_duree_stage):
				g.add((stage3, MT.duree, Literal(int(row_stage.stages_m2_alternance_duree_stage.strip(" mois")))))
			
			if pd.notna(row_stage.stages_m2_alternance_type_org_stage):
				g.add((stage3, MT.nomOrganismeAcceuil, Literal(row_stage.stages_m2_alternance_type_org_stage)))
			
			if pd.notna(row_stage.stages_m2_alternance_missions_stage):
				g.add((stage3, MT.sujetMissions, Literal(row_stage.stages_m2_alternance_missions_stage)))
			
		if pd.notna(row_stage.id_m_2_tetrarechim):
			stage4 = URIRef(MT + (row_stage.id_m_2_tetrarechim).replace(" ", ""))
			g.add((etudiant, MT.aCommeExperience, stage4))	
			g.add((stage4, RDF.type, MT.Stage))
			if pd.notna(row_stage.stages_m2_tetrarechim_duree):
				g.add((stage4, MT.duree, Literal(int(row_stage.stages_m2_tetrarechim_duree.strip(" mois")))))
			if pd.notna(row_stage.stages_m2_tetrarechim_nom_orga):
				g.add((stage4, MT.nomOrganismeAcceuil, Literal(row_stage.stages_m2_tetrarechim_nom_orga)))
			if pd.notna(row_stage.stages_m2_tetrarechim_type_orga):
				g.add((stage4, MT.typeOrganisme, Literal(row_stage.stages_m2_tetrarechim_type_orga)))
			if pd.notna(row_stage.stages_m2_tetrarechim_sujet):
				g.add((stage4, MT.sujetMissions, Literal(row_stage.stages_m2_tetrarechim_sujet)))
			if pd.notna(row_stage.stages_m2_tetrarechim_tuteur):
				tuteurs = row_stage.stages_m2_tetrarechim_tuteur.split(", ")
				for tuteur in tuteurs:
					nom_prenom = tuteur.strip(" ").split(" ") 
					tuteur = nom_prenom[0] + "_" + "_".join([nom.upper() for nom in nom_prenom[1:]])
					print(tuteur)
					g.add((etudiant, MT.aPourTuteurDeStage, URIRef(MT + tuteur)))
				



	# print("--- printing mboxes ---")
	# print(g.serialize(format='n3'))
    

	return g

g = ajout_stage(df_stage, g)


Sylvain_KAHANE
Sylvain_KAHANE
Sylvain_KAHANE
Loïc_GROBOL
Delphine_BATTISTELLI
Loïc_GROBOL
Delphine_BATTISTELLI
Sarra_EL_AYARI
Sam_BIGEARD
Sylvain_KAHANE
Sylvain_KAHANE
Sylvain_KAHANE
Delphine_BATTISTELLI
Pierre_MAGISTRY
Cédric_GENDROT
Cédric_GENDROT
Guillaume_WISNIEWSKI
Pierre_MAGISTRY
Pierre_MAGISTRY


In [8]:

def ajout_parcours(df_parcours, g):
    '''On ajoute les parcours à notre ontologie'''
    for row_parcours in df_parcours.itertuples(index=False, name = 'Parcours'):
        # On crée un URI pour chaque parcours
        parcours = URIRef(MT + (row_parcours.nom_parcours).replace(" ", ""))
        # On ajoute le parcours à notre ontologie
        if "M1" in row_parcours.nom_parcours:
            g.add((parcours, RDF.type, MT.M1))
        elif "M2" in row_parcours.nom_parcours:
            if "Alterance" in row_parcours.nom_parcours:
                g.add((parcours, RDF.type, MT.Alternance))
            if "RD" in row_parcours.nom_parcours:
                g.add((parcours, RDF.type, MT.RD))
            if "IM" in row_parcours.nom_parcours:
                g.add((parcours, RDF.type, MT.IM))
            if "TetraDom" in row_parcours.nom_parcours:
                g.add((parcours, RDF.type, MT.TetraDom))
        
        g.add((parcours, MT.etablissement, URIRef(MT+ row_parcours.etablissement)))
        g.add((parcours, MT.annee, Literal(row_parcours.annee)))
        
        # PAS BESOIN ON SAIT avec leur df à eux
        # for eleve in row_parcours.liste_eleves:
        #     g.add((parcours, MT.aPourEleve, URIRef(MT + eleve)))
    
    print("--- printing mboxes ---")
    print(g.serialize(format='n3'))
    
    return g


g = ajout_parcours(df_parcours, g)

--- printing mboxes ---
@prefix mt: <http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL> a owl:Ontology .

mt:Ashley_RATIER a mt:Eleve ;
    mt:autre_master "Maitrise en MEEF, Université de Cergy-Pontoise, Antony" ;
    mt:bac "S" ;
    mt:dateNaissance "24/11/1998" ;
    mt:difficulte_M1 4 ;
    mt:difficulte_M2 4 ;
    mt:genre "Femme" ;
    mt:inscritA mt:M1_Nanterre_23-24,
        mt:M2_Alterance_24-25 ;
    mt:licence "Licence en Science du Langage, Université d’Orléans" ;
    mt:mention_licence "Sciences du langage" ;
    mt:nationalite "FR" ;
    mt:nom "RATIER" ;
    mt:prefereCommeEvaluation mt:DevoirMaisonSolo,
        mt:ProjetSolo ;
    mt:prenom "Ashley" ;
    mt:quota_pleure "4/mois" ;
    mt:recommendation "À VOTRE ENNEMI JURÉ !" ;
   

In [9]:
def ajout_etablissement(g): 
    g.add((URIRef(MT+'Inalco'), RDF.type, MT.Etablissement))
    g.add((URIRef(MT+'Nanterre'), RDF.type, MT.Etablissement))
    g.add((URIRef(MT+'SorbonneNouvelle'), RDF.type, MT.Etablissement))
    return g

g = ajout_etablissement(g)
print(g.serialize(format='n3'))

@prefix mt: <http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL> a owl:Ontology .

mt:Ashley_RATIER a mt:Eleve ;
    mt:autre_master "Maitrise en MEEF, Université de Cergy-Pontoise, Antony" ;
    mt:bac "S" ;
    mt:dateNaissance "24/11/1998" ;
    mt:difficulte_M1 4 ;
    mt:difficulte_M2 4 ;
    mt:genre "Femme" ;
    mt:inscritA mt:M1_Nanterre_23-24,
        mt:M2_Alterance_24-25 ;
    mt:licence "Licence en Science du Langage, Université d’Orléans" ;
    mt:mention_licence "Sciences du langage" ;
    mt:nationalite "FR" ;
    mt:nom "RATIER" ;
    mt:prefereCommeEvaluation mt:DevoirMaisonSolo,
        mt:ProjetSolo ;
    mt:prenom "Ashley" ;
    mt:quota_pleure "4/mois" ;
    mt:recommendation "À VOTRE ENNEMI JURÉ !" ;
    mt:redoublement false ;

In [10]:
def ajout_prof(df_prof, g, df_cours):
	'''On ajoute les profs à notre ontologie'''
	for row_prof in df_prof.itertuples(index=False, name = 'Prof'):
		prof = URIRef(MT + row_prof.id_prof)
		g.add((prof, RDF.type, MT.Enseignant))
		g.add((prof, MT.nom, Literal(row_prof.nom_prof)))
		g.add((prof, MT.prenom, Literal(row_prof.prenom_prof)))
		g.add((prof, MT.genre, Literal(row_prof.genre)))
		g.add((prof, MT.employePar, URIRef(MT + row_prof.etablissement)))


# LES COURS ENSEIGNEPAR mais relation inverse
		# cours = (row_prof.cours).replace('[', '').replace(']','').split(', ')
		# print(cours)
		# for c in cours:
		# 	if c != "":
		# 		print(c.strip('"').strip("'"))
		# 		if c.strip('"').strip("'") in df_cours['nom_cours'].values: 
		# 			id_cours = df_cours.loc[df_cours['nom_cours'] == c.strip('"').strip("'"), 'id_cours'].iloc[0]
		# 			g.add((prof, MT.enseigne, URIRef(MT + id_cours)))
		# 		else:
		# 			print(f"WTF: {c}")

# METTRE ID des etudiants à la place des noms dans la df_prof 
		# if pd.notna(row_prof.directeur_memoire_de):
		# 	encadre = (row_prof.directeur_memoire_de).replace('[', '').replace(']','').split(', ')
		# 	for e in encadre:
		# 		# g.add((prof, MT.encadre, URIRef(MT + e))

	print("--- printing mboxes ---")
	print(g.serialize(format='n3'))

	return g
	
g = ajout_prof(df_prof, g, df_cours)

--- printing mboxes ---
@prefix mt: <http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL> a owl:Ontology .

mt:Alexandre_ROULOIS a mt:Enseignant ;
    mt:employePar mt:Inalco ;
    mt:genre "Homme" ;
    mt:nom "ROULOIS" ;
    mt:prenom "Alexandre" .

mt:Armand_STRICKER a mt:Enseignant ;
    mt:employePar mt:Inalco ;
    mt:genre "Homme" ;
    mt:nom "STRICKER" ;
    mt:prenom "Armand" .

mt:Ashley_RATIER a mt:Eleve ;
    mt:autre_master "Maitrise en MEEF, Université de Cergy-Pontoise, Antony" ;
    mt:bac "S" ;
    mt:dateNaissance "24/11/1998" ;
    mt:difficulte_M1 4 ;
    mt:difficulte_M2 4 ;
    mt:genre "Femme" ;
    mt:inscritA mt:M1_Nanterre_23-24,
        mt:M2_Alterance_24-25 ;
    mt:licence "Licence en Science du Langage, Université d’Orlé

In [11]:
def ajout_cours(df_cours, g):
	'''On ajoute les cours à notre ontologie'''
	for row_cours in df_cours.itertuples(index=False, name = 'Cours'):
		# On crée un URI pour chaque cours
		cours = URIRef(MT + row_cours.id_cours)
		# On ajoute le cours à notre ontologie
		g.add((cours, RDF.type, MT.Cours))
		g.add((cours, MT.nom_cours, Literal(row_cours.nom_cours)))
		if pd.notna(row_cours.id_prof):
			for prof in row_cours.id_prof.replace("[", '').replace("]", '').split(', '):
				# print(prof.strip("'"))
				g.add((cours, MT.dispensePar, URIRef(MT + prof.strip("'"))))
		g.add((cours, MT.enseigneA, URIRef(MT + row_cours.etablissement)))
		g.add((cours, MT.annee, Literal(row_cours.annee)))
		
		# RAJOUTE LES MODALITES D'EVALUATION
		# g.add((cours, MT.evalueAvec, URIRef(MT+row_cours.evaluation)))

	print("--- printing mboxes ---")
	print(g.serialize(format='n3'))
	
	return g
g = ajout_cours(df_cours, g)


--- printing mboxes ---
@prefix mt: <http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

<http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL> a owl:Ontology .

mt:Ashley_RATIER a mt:Eleve ;
    mt:autre_master "Maitrise en MEEF, Université de Cergy-Pontoise, Antony" ;
    mt:bac "S" ;
    mt:dateNaissance "24/11/1998" ;
    mt:difficulte_M1 4 ;
    mt:difficulte_M2 4 ;
    mt:genre "Femme" ;
    mt:inscritA mt:M1_Nanterre_23-24,
        mt:M2_Alterance_24-25 ;
    mt:licence "Licence en Science du Langage, Université d’Orléans" ;
    mt:mention_licence "Sciences du langage" ;
    mt:nationalite "FR" ;
    mt:nom "RATIER" ;
    mt:prefereCommeEvaluation mt:DevoirMaisonSolo,
        mt:ProjetSolo ;
    mt:prenom "Ashley" ;
    mt:quota_pleure "4/mois" ;
    mt:recommendation "À VOTRE ENNEMI JURÉ !" ;
   

In [12]:
# def ajout_evaluation(dico_evaluation, g):
# 	'''On ajoute les types d'évaluation à notre ontologie'''
# 	for key, value in dico_evaluation.items():
# 		evaluation = URIRef(MT + key)
# 		g.add((evaluation, RDF.type, MT.Evaluation))
# 		# g.add((evaluation, MT.nom, Literal(value)))	
# 	print("--- printing mboxes ---")
# 	print(g.serialize(format='n3'))
	
# 	return g
# ajout_evaluation(dico_evaluation, g)

In [13]:
g.serialize(destination="../Master_TAL.rdf",format="xml")


<Graph identifier=Ne0226eb944f347f084ba7e3fb93a35ff (<class 'rdflib.graph.Graph'>)>

In [117]:
def get_predicats(g):
	'''Choppe du prédicat pour voir les noms de relations utilisés dans l'ontologie
	et genre si il y en a qu'on a oublié'''
	predicats = []
	for s, p, o in g.triples((None, None, None)):
		predicats.append(p)
	return set(predicats)

print(get_predicats(g))

{rdflib.term.URIRef('http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#temps_sommeil'), rdflib.term.URIRef('http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#prefereCommeEvaluation'), rdflib.term.URIRef('http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#etablissement'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'), rdflib.term.URIRef('http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#suitLeCoursDe'), rdflib.term.URIRef('http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#nom_cours'), rdflib.term.URIRef('http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#prenom'), rdflib.term.URIRef('http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#relation_mac'), rdflib.term.URIRef('http://www.semanticweb.org/tacitus/ontologies/2024/11/MASTER_TAL#annee'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subPropertyOf'), rdflib.term.URIRef('http://www.semanticweb.org/tacitu

In [118]:
def get_classes(g):
	'''	Choppe tous les types de classes de onto '''
	classes = []
	for s, p, o in g.triples((None, RDF.type, None)):
		classes.append(o)
	return len(set(classes))

print(get_classes(g))

15
